In [ ]:
!pip install torch transformers opencv-python pillow

In [ ]:
!pip install ultralytics

In [ ]:
import cv2
import torch
from transformers import TrOCRProcessor, VisionEncoderDecoderModel
from PIL import Image as PILImage
import numpy as np
from ultralytics import YOLO
from google.colab.patches import cv2_imshow # Import cv2_imshow from google.colab.patches

# Load YOLOv8 model using torch
model=YOLO('/content/best.pt')

# Load TrOCR model and processor for OCR
processor = TrOCRProcessor.from_pretrained("microsoft/trocr-base-handwritten")
ocr_model = VisionEncoderDecoderModel.from_pretrained("microsoft/trocr-base-handwritten")

video_file = "/content/output_video.mp4"  # Path to your video file
output_file = "/content/results.txt"

# Preprocess image for OCR model (convert OpenCV image to PIL)
def preprocess_for_ocr(plate_img):
    # Convert OpenCV Mat (BGR) to RGB format and then to PIL image
    pil_img = PILImage.fromarray(cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB))
    return pil_img

# Decode the OCR predictions
def decode_predictions(ocr_inputs):
    pixel_values = processor(images=ocr_inputs, return_tensors="pt").pixel_values
    generated_ids = ocr_model.generate(pixel_values)
    plate_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return plate_text.strip()

cap = cv2.VideoCapture(video_file)
if not cap.isOpened():
    print("Error opening video file")
    exit()

frame_number = 0
with open(output_file, 'w') as writer:
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_number += 1

        # Convert frame to RGB and resize to 640x640 for YOLO model
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = model(frame_rgb)

        # Get detected bounding boxes
        boxes = results[0].boxes  # Extract the boxes object
        for box in boxes:
            # Extract coordinates and confidence from the box
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
            conf = box.conf.item()
            class_id = box.cls.item()

            # Crop the detected license plate region
            plate_img = frame[y1:y2, x1:x2]

            # Preprocess the plate image for the OCR model
            preprocessed_plate = preprocess_for_ocr(plate_img)

            # Perform OCR using the TrOCR model
            plate_text = decode_predictions(preprocessed_plate)

            # Write the result to the output file
            writer.write(f"Frame {frame_number}: Detected plate - {plate_text}\n")

            # Draw the bounding box and detected text on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(frame, plate_text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        # Display the frame with the detected bounding boxes
        cv2_imshow(frame) # Use cv2_imshow instead of cv2.imshow
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()